In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
#import geopandas as gpd

In [12]:
url = f'modelling_data_to_check.csv'
df = pd.read_csv(url, parse_dates=['month_year'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186374 entries, 0 to 186373
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   org_code            186374 non-null  object        
 1   region_name         186374 non-null  object        
 2   org_type            186374 non-null  object        
 3   staff_group         186374 non-null  object        
 4   month_year          186374 non-null  datetime64[ns]
 5   leaver_rate         186374 non-null  float64       
 6   joiner_rate         186374 non-null  float64       
 7   total_sip_FTE       186374 non-null  float64       
 8   local_unemployment  186374 non-null  float64       
 9   sickness_absence    186374 non-null  float64       
 10  anxiety_stress      186374 non-null  float64       
 11  asthma              186374 non-null  float64       
 12  back_problems       186374 non-null  float64       
 13  blood_disorder      186374 no

Specify the dependent variable (dv) and independent variables (ivs). All other fields to be dropped. 

In [17]:
dv = 'leaver_rate'
ivs = ['local_unemployment','sickness_absence','joiner_rate','total_sip_FTE','anxiety_stress']
to_drop = ['month_year','org_code','region_name','org_type','staff_group', dv]

Define the design matrix (X) and the dependent variable (y)


In [18]:
df.reset_index(drop = True)
X = df.drop(to_drop, axis=1)
y = df[dv]

y.head()


0    0.012726
1    0.012726
2    0.000000
3    0.025540
4    0.025540
Name: leaver_rate, dtype: float64

In [19]:

# Add a constant column to the design matrix
X = sm.add_constant(X)

X.tail()

# Convert True/False categories to integer 0/1
#bool_columns = X.select_dtypes(include='bool').columns
#X[bool_columns] = X[bool_columns].astype(int)


,const,joiner_rate,total_sip_FTE,local_unemployment,sickness_absence,anxiety_stress,asthma,back_problems,blood_disorder,dental,...,headache_mig,infectious_dis,nervous_system,other,other_msk,pregnancy_related,respiratory,skin_disorders,substance_abus,unknown
186369,1.0,0.029474,5900.50194,0.047,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186370,1.0,0.000000,5427.83895,0.040,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186371,1.0,0.000000,9218.21028,0.047,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186372,1.0,0.013631,12984.91227,0.040,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186373,1.0,0.030986,5391.82390,0.047,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Fit the regression model
model = sm.OLS(y, X)
results = model.fit()

# Print the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            leaver_rate   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     80.23
Date:                Mon, 24 Jul 2023   Prob (F-statistic):               0.00
Time:                        16:38:12   Log-Likelihood:             3.0586e+05
No. Observations:              186374   AIC:                        -6.117e+05
Df Residuals:                  186350   BIC:                        -6.114e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0179      0